<a href="https://www.kaggle.com/code/noelwilliam/deep-fake-video-classification?scriptVersionId=162968422" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing necessary libraries

In [1]:
pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.0 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import pathlib
import cv2
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from mtcnn import MTCNN
import os
import shutil
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Downloading and extraction our training data

In [ ]:
!wget https://zenodo.org/record/4068245/files/DeepfakeTIMIT.tar.gz?download=1

In [ ]:
!tar -xf ./DeepfakeTIMIT.tar.gz?download=1

In [ ]:
!wget https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip

In [ ]:
!unzip -qq ./VidTIMIT.zip

In [ ]:
!rm -rf ./__MACOSX

##Making necessary directories

In [ ]:
!mkdir -p ./Training\ Videos/preproc_real_vids
!mkdir -p ./Training\ Videos/preproc_fake_vids

In [ ]:
!mkdir -p ./Testing\ Videos/preproc_real_vids
!mkdir -p ./Testing\ Videos/preproc_fake_vids

In [ ]:
!mkdir ./Remaining\ Real\ Videos

In [ ]:
real_fake_vids_paths = ["./DeepfakeTIMIT/higher_quality","./VidTIMIT"]

## Analyzing Training data
Making fake and real videos equal in quantity.

In [ ]:
for base_vid_path in real_fake_vids_paths:

    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):

        vid_counter += 1

    print("Total Number of Videos are {}".format(vid_counter))
    vid_counter = 0

In [ ]:
vid_counter = 0

for single_vid_path in pathlib.Path(real_fake_vids_paths[1]).glob("*/*.avi"):

    if str(single_vid_path.parts[-2]) in os.listdir(real_fake_vids_paths[0]):
        vid_counter += 1

    else:
        if not os.path.isdir("./Remaining Real Videos/"+single_vid_path.parts[-2]):
            os.mkdir("./Remaining Real Videos/"+single_vid_path.parts[-2])

        shutil.move(src=single_vid_path,
                    dst="./Remaining Real Videos/"+"/".join([single_vid_path.parts[-2],
                                                           single_vid_path.parts[-1]]))

print("Total Number of Videos are {}".format(vid_counter))

In [ ]:
for base_vid_path in real_fake_vids_paths:

    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):

        vid_counter += 1

    print("Total Number of Videos are {}".format(vid_counter))

# Functions to preprocess training videos

In [ ]:
def detect_face(frame):

    face_detector = MTCNN()
    face_attr = face_detector.detect_faces(frame)

    if len(face_attr) > 0:
        if len(face_attr[0]['box']) > 0:
            return face_attr[0]['box'], face_attr[0]['keypoints']['left_eye'], face_attr[0]['keypoints']['right_eye']
    else:
        return [None]*3

In [ ]:
def determine_rotation_angle(left_eye_kp, right_eye_kp):

    delta_y = right_eye_kp[1] - left_eye_kp[1]
    delta_x = right_eye_kp[0] - left_eye_kp[0]

    return np.arctan(delta_y/delta_x)

In [ ]:
def determine_scaling_factor(rescaled_img_size,left_eye_kp,right_eye_kp):

    left_eye_kp = np.array(left_eye_kp)
    right_eye_kp = np.array(right_eye_kp)

    whole_frame_eye_dist = np.linalg.norm(x=(right_eye_kp - left_eye_kp),ord=2)

    cropped_left_eye_kp = np.array([0.47*rescaled_img_size[0], 0.32*rescaled_img_size[1]])
    cropped_right_eye_kp = np.array([0.47*rescaled_img_size[0], 0.68*rescaled_img_size[1]])

    cropped_frame_eye_dist = np.linalg.norm(x=(cropped_right_eye_kp - cropped_left_eye_kp),
                                            ord=2)

    scaling_factor = cropped_frame_eye_dist/whole_frame_eye_dist

    return scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp

In [ ]:
def determine_rotation_center(left_eye_kp, right_eye_kp):

    center_x = (left_eye_kp[0] + right_eye_kp[0])/2
    center_y = (left_eye_kp[1] + right_eye_kp[1])/2

    return (center_x, center_y)

### Optional Step
Testing Our functions

In [ ]:
'''
single_vid_path = pathlib.Path("./DeepfakeTIMIT/higher_quality/mrgg0/si569-video-msjs1.avi")

print("\n\nGoing to preprocess Video at {}".format(single_vid_path))

video = cv2.VideoCapture(str(single_vid_path))
cropped_aligned_faces_list = list()

while True:

    is_frame, frame = video.read()

    if not is_frame:
        break

    print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],
                                                                  frame.shape))

    face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)

    if face_bbox == None:
        continue

    rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);
    scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                            left_eye_kp, right_eye_kp)
    rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

    #print("Computed everything to get the Rotation Matrix for the Frame")

    rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                         angle=rotation_angle,scale=scaling_factor)

    #print("Computed the Rotation Matrix for the Frame")

    cropped_center_x = cropped_left_eye_kp[0]
    cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
    cropped_center = (cropped_center_x, cropped_center_y)

    rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
    rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

    #print("Translated the center in Rotation Matrix to the Cropped Frame Center")

    cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                      cv2.INTER_CUBIC)

    #print("Cropped and Aligned the face in the Frame")

    cropped_aligned_faces_list.append(cropped_aligned_face)

    print("Preprocessed Frame of {}".format(single_vid_path.parts[-1]))

    plt.imshow(cropped_aligned_face)

    break;

video.release()

cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)


if "DeepfakeTIMIT" in list(single_vid_path.parts):
    np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)
else:
    np.savez("./Training Videos/preproc_real_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)

print("\n\nProcessed Video at {}".format(single_vid_path))
'''

In [ ]:
"""
def preproc_single_vid(single_vid_path):

    print("\n\nGoing to preprocess Video at {}\n".format(single_vid_path))

    video = cv2.VideoCapture(str(single_vid_path))
    cropped_aligned_faces_list = list()

    while True:

        is_frame, frame = video.read()

        if not is_frame:
            break

        print("Going to preprocess frame of {}".format(single_vid_path.parts[-1]))

        face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)
        angle_bw_eyes = determine_rotation_angle(left_eye_kp, right_eye_kp);
        scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                        left_eye_kp, right_eye_kp)
        rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

        rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                     angle=rotation_angle,scale=scaling_factor)

        cropped_center_x = cropped_left_eye_kp[0]
        cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
        cropped_center = (cropped_center_x, cropped_center_y)

        rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
        rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

        cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                  cv2.INTER_CUBIC)

        cropped_aligned_faces_list.append(cropped_aligned_face)

    video.release()

    cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

    if "DeepfakeTIMIT" in list(single_vid_path.parts):
        np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(single_vid_path.parts[-1]),
                         args=cropped_aligned_faces_list)
    else:
        np.savez("./Training Videos/preproc_real_vids/{}.npz".format(single_vid_path.parts[-1]),
                         args=cropped_aligned_faces_list)

    print("\n\nProcessed Video at {}".format(single_vid_path))
"""

#Function to preprocess a single video
This function uses all the function made previously.

In [ ]:
def preproc_single_vid(mb_vids_paths):

    for single_vid_path in mb_vids_paths:

        print("\n\nGoing to preprocess Video at {}".format(single_vid_path))

        video = cv2.VideoCapture(str(single_vid_path))
        cropped_aligned_faces_list = list()

        while True:

            is_frame, frame = video.read()

            if not is_frame:
                break
            """
            print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],
                                                                  frame.shape))
            """
            face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)

            if face_bbox == None:
                continue

            rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);
            scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                            left_eye_kp, right_eye_kp)
            rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

            #print("Computed everything to get the Rotation Matrix for the Frame")

            rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                         angle=rotation_angle,scale=scaling_factor)

            #print("Computed the Rotation Matrix for the Frame")

            cropped_center_x = cropped_left_eye_kp[0]
            cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
            cropped_center = (cropped_center_x, cropped_center_y)

            rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
            rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

            #print("Translated the center in Rotation Matrix to the Cropped Frame Center")

            cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                      cv2.INTER_CUBIC)

            #print("Cropped and Aligned the face in the Frame")

            cropped_aligned_faces_list.append(cropped_aligned_face)

            print("Preprocessed Frame of {}".format(single_vid_path.parts[-1]))

        video.release()

        cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

        if "DeepfakeTIMIT" in list(single_vid_path.parts):
            np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)
        else:
            np.savez("./Training Videos/preproc_real_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)

        print("\n\nProcessed Video at {}".format(single_vid_path))

# Preprocessing all videos

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from collections import deque

vid_counter = 0
mb_number = 0

real_fake_preproc_vids_paths = ["./Training Videos/preproc_fake_vids",
                               "./Training Videos/preproc_real_vids"]

for vids_path, preproc_vids_path in zip(real_fake_vids_paths,
                                                    real_fake_preproc_vids_paths):

    vids2preproc_path_list = deque()

    for single_vid_path in pathlib.Path(vids_path).glob("*/*.avi"):

        vids2preproc_path_list.append(single_vid_path)
        vid_counter += 1

        if vid_counter % (os.cpu_count()) == 0:

            start_time = time.time()

            with ThreadPoolExecutor(max_workers=(os.cpu_count())) as pool:
                future = pool.submit(preproc_single_vid, list(vids2preproc_path_list))
                future.result()

            end_time = time.time()
            mb_number += 1
            elapsed_time = end_time - start_time

            print("\n\nProcessed Mini Batch # {} of 64 Videos in {} seconds".format(mb_number,elapsed_time))

            vids2preproc_path_list.clear()


# Storing Training Videos
Seperately storing training fake and real videos in seperate list or numpy arrays

In [3]:
training_real_vids_paths = list()
training_fake_vids_paths = list()

for single_img_path in pathlib.Path("/kaggle/input/deep-fake-and-real-videos/Training Videos").glob("*/*.npz"):

    if "real" in str(single_img_path.parts[-2]).split("_"):
        training_real_vids_paths.append(str(single_img_path))

    elif "fake" in str(single_img_path.parts[-2]).split("_"):
        training_fake_vids_paths.append(str(single_img_path))

training_real_vids_paths = np.array(training_real_vids_paths)
training_fake_vids_paths = np.array(training_fake_vids_paths)

In [4]:
len(training_real_vids_paths)

197

In [5]:
len(training_fake_vids_paths)

320

In [6]:
# Calculating how many times to repeat the real videos
repeat_times = len(training_fake_vids_paths) // len(training_real_vids_paths)

# Repeat the real videos
oversampled_real_vids_paths = np.repeat(training_real_vids_paths, repeat_times)

# If there are still fewer real videos than fake videos, append some real videos to make up the difference
remainder = len(training_fake_vids_paths) - len(oversampled_real_vids_paths)
if remainder > 0:
    oversampled_real_vids_paths = np.concatenate((oversampled_real_vids_paths, training_real_vids_paths[:remainder]))

# Now oversampled_real_vids_paths should have the same length as training_fake_vids_paths
assert len(oversampled_real_vids_paths) == len(training_fake_vids_paths)

In [7]:
training_real_vids_paths = oversampled_real_vids_paths
len(training_real_vids_paths)

320

#Training Data Generator

In [8]:
''' 
def custom_training_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):
    for i in range(len(training_fake_vids_paths) // 2):
        X_train_mb = []
        Y_train_mb = []

        training_fake_mb_vids_paths = training_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]
        training_real_mb_vids_paths = training_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]

        for single_fake_vid_path, single_real_vid_path in zip(training_fake_mb_vids_paths,
                                                              training_real_mb_vids_paths):

            fake_frames = np.load(file=single_fake_vid_path)['args']
            # Setting image to (224, 224) if not already
            fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]
            fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))
            fake_frames_resized = fake_frames_resized[:max_frames]  # This ensures 100 frames


            real_frames = np.load(file=single_real_vid_path)['args']

            real_frames_resized = [cv2.resize(frame, target_frame_size) for frame in real_frames]
            real_frames_resized += [np.zeros_like(real_frames_resized[0])] * (max_frames - len(real_frames_resized))
            real_frames_resized = real_frames_resized[:max_frames]

            X_train_mb.append(real_frames_resized)
            X_train_mb.append(fake_frames_resized)

            # Real labeled as 0 and Fake labeled as 1
            Y_train_mb.append(0)
            Y_train_mb.append(1)

        X_train_mb = np.array(X_train_mb)
        Y_train_mb = np.array(Y_train_mb)

        yield X_train_mb, Y_train_mb
    '''

" \ndef custom_training_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):\n    for i in range(len(training_fake_vids_paths) // 2):\n        X_train_mb = []\n        Y_train_mb = []\n\n        training_fake_mb_vids_paths = training_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]\n        training_real_mb_vids_paths = training_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]\n\n        for single_fake_vid_path, single_real_vid_path in zip(training_fake_mb_vids_paths,\n                                                              training_real_mb_vids_paths):\n\n            fake_frames = np.load(file=single_fake_vid_path)['args']\n            # Setting image to (224, 224) if not already\n            fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]\n            fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))\n            fake_frames_resized = fake_fram

In [9]:
def custom_training_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):
    while True:  #works like repeat()
        for i in range(len(training_fake_vids_paths) // 2):
            X_train_mb = []
            Y_train_mb = []

            training_fake_mb_vids_paths = training_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]
            training_real_mb_vids_paths = training_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]

            for single_fake_vid_path, single_real_vid_path in zip(training_fake_mb_vids_paths,
                                                                  training_real_mb_vids_paths):

                fake_frames = np.load(file=single_fake_vid_path)['args']
                # Setting image to (224, 224) if not already
                fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]
                fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))
                fake_frames_resized = fake_frames_resized[:max_frames]  # This ensures 100 frames


                real_frames = np.load(file=single_real_vid_path)['args']

                real_frames_resized = [cv2.resize(frame, target_frame_size) for frame in real_frames]
                real_frames_resized += [np.zeros_like(real_frames_resized[0])] * (max_frames - len(real_frames_resized))
                real_frames_resized = real_frames_resized[:max_frames]

                X_train_mb.append(real_frames_resized)
                X_train_mb.append(fake_frames_resized)

                # Real labeled as 0 and Fake labeled as 1
                Y_train_mb.append(0)
                Y_train_mb.append(1)

            X_train_mb = np.array(X_train_mb)
            Y_train_mb = np.array(Y_train_mb)

            yield X_train_mb, Y_train_mb

# Split training data for validation data
Creating seperate data for validation same as training data

In [10]:
from sklearn.model_selection import train_test_split

# Combine real and fake paths
all_vids_paths = np.concatenate((training_real_vids_paths, training_fake_vids_paths))

# Create labels: 0 for real and 1 for fake
all_vids_labels = np.concatenate((np.zeros(len(training_real_vids_paths)), np.ones(len(training_fake_vids_paths))))

# Split the data into training and validation sets (80-20 split here)
train_vids_paths, val_vids_paths, train_vids_labels, val_vids_labels = train_test_split(all_vids_paths, all_vids_labels, test_size=0.2, random_state=42)

# Separate real and fake validation video paths
validation_real_vids_paths = val_vids_paths[val_vids_labels == 0]
validation_fake_vids_paths = val_vids_paths[val_vids_labels == 1]


#Validation Data Generator

In [11]:
      '''
        def custom_validation_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):
    for i in range(len(validation_fake_vids_paths) // 2):
        X_val_mb = []
        Y_val_mb = []

        validation_fake_mb_vids_paths = validation_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]
        validation_real_mb_vids_paths = validation_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]

        for single_fake_vid_path, single_real_vid_path in zip(validation_fake_mb_vids_paths,
                                                              validation_real_mb_vids_paths):

            fake_frames = np.load(file=single_fake_vid_path)['args']
            # Setting image to (224, 224) if not already
            fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]
            fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))
            fake_frames_resized = fake_frames_resized[:max_frames]  # This ensures 100 frames


            real_frames = np.load(file=single_real_vid_path)['args']

            real_frames_resized = [cv2.resize(frame, target_frame_size) for frame in real_frames]
            real_frames_resized += [np.zeros_like(real_frames_resized[0])] * (max_frames - len(real_frames_resized))
            real_frames_resized = real_frames_resized[:max_frames]

            X_val_mb.append(real_frames_resized)
            X_val_mb.append(fake_frames_resized)

            # Real labeled as 0 and Fake labeled as 1
            Y_val_mb.append(0)
            Y_val_mb.append(1)

        X_val_mb = np.array(X_val_mb)
        Y_val_mb = np.array(Y_val_mb)

        yield X_val_mb, Y_val_mb
        
        '''

"\n  def custom_validation_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):\n    for i in range(len(validation_fake_vids_paths) // 2):\n  X_val_mb = []\n  Y_val_mb = []\n\n  validation_fake_mb_vids_paths = validation_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]\n  validation_real_mb_vids_paths = validation_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]\n\n  for single_fake_vid_path, single_real_vid_path in zip(validation_fake_mb_vids_paths,\n                                                        validation_real_mb_vids_paths):\n\n      fake_frames = np.load(file=single_fake_vid_path)['args']\n      # Setting image to (224, 224) if not already\n      fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]\n      fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))\n      fake_frames_resized = fake_frames_resized[:max_frames]  # This ensures 100 frames\n\

In [12]:
def custom_validation_data_generator(mb_size, target_frame_size=(224, 224), max_frames=100):
    while True:  # works like repeat()
        for i in range(len(validation_fake_vids_paths) // 2):
            X_val_mb = []
            Y_val_mb = []

            validation_fake_mb_vids_paths = validation_fake_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]
            validation_real_mb_vids_paths = validation_real_vids_paths[i * (mb_size // 2):(i + 1) * (mb_size // 2)]

            for single_fake_vid_path, single_real_vid_path in zip(validation_fake_mb_vids_paths,
                                                                  validation_real_mb_vids_paths):

                fake_frames = np.load(file=single_fake_vid_path)['args']
                # Setting image to (224, 224) if not already
                fake_frames_resized = [cv2.resize(frame, target_frame_size) for frame in fake_frames]
                fake_frames_resized += [np.zeros_like(fake_frames_resized[0])] * (max_frames - len(fake_frames_resized))
                fake_frames_resized = fake_frames_resized[:max_frames]  # This ensures 100 frames


                real_frames = np.load(file=single_real_vid_path)['args']

                real_frames_resized = [cv2.resize(frame, target_frame_size) for frame in real_frames]
                real_frames_resized += [np.zeros_like(real_frames_resized[0])] * (max_frames - len(real_frames_resized))
                real_frames_resized = real_frames_resized[:max_frames]

                X_val_mb.append(real_frames_resized)
                X_val_mb.append(fake_frames_resized)

                # Real labeled as 0 and Fake labeled as 1
                Y_val_mb.append(0)
                Y_val_mb.append(1)

            X_val_mb = np.array(X_val_mb)
            Y_val_mb = np.array(Y_val_mb)

            yield X_val_mb, Y_val_mb

#Initializing data Generators

Initializing our data generator and checking shape of data for our model

In [13]:
train_datagen = custom_training_data_generator(4)

In [14]:
val_datagen = custom_validation_data_generator(4)

In [15]:
X_train_mb, Y_train_mb = next(train_datagen)

In [16]:
X_train_mb.shape

(4, 100, 224, 224, 3)

In [17]:
Y_train_mb.shape

(4,)

# Neural Network

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Flatten, Dropout, Input
from tensorflow.keras.applications import ResNet50

In [20]:
def create_model():

    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    inputs = Input(shape=(100, 224, 224, 3))

    x = TimeDistributed(base_model)(inputs)

    x = TimeDistributed(Flatten())(x)

    x = LSTM(50, return_sequences = True)(x)
    
    x = LSTM(50, return_sequences = True)(x)
    
    x = LSTM(50, return_sequences = False)(x)

    x = Dropout(0.25)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [21]:
model = create_model()

94765736/94765736 [==============================] - 0s 0us/step


In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 224, 224, 3  0         
                             )]                                  
                                                                 
 time_distributed (TimeDistr  (None, 100, 7, 7, 2048)  23587712  
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 100, 100352)      0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 100, 50)           20080600  
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                             

#Fitting Our Model

In [24]:
train_steps_per_epoch = len(train_vids_paths) // 4  # mini-batch size is 4
val_steps = len(val_vids_paths) // 4

In [25]:
model.fit(train_datagen, steps_per_epoch=train_steps_per_epoch,
                    validation_data=val_datagen, validation_steps=val_steps,
                    epochs=10)

Epoch 1/10
128/128 [==============================] - 218s 2s/step - loss: 0.6947 - accuracy: 0.5254 - val_loss: 0.6912 - val_accuracy: 0.5703
Epoch 2/10
128/128 [==============================] - 196s 2s/step - loss: 0.6925 - accuracy: 0.5137 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 3/10
128/128 [==============================] - 196s 2s/step - loss: 0.7005 - accuracy: 0.4824 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/10
128/128 [==============================] - 196s 2s/step - loss: 0.6992 - accuracy: 0.5020 - val_loss: 0.6929 - val_accuracy: 0.5078
Epoch 5/10
128/128 [==============================] - 196s 2s/step - loss: 0.6911 - accuracy: 0.5332 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
128/128 [==============================] - 196s 2s/step - loss: 0.6944 - accuracy: 0.5078 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 7/10
128/128 [==============================] - 196s 2s/step - loss: 0.6981 - accuracy: 0.4902 - val_loss: 0.6934 - val_accuracy: 0.5000